**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

df = pd.read_csv('ratings.csv')
df.head(3)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182


In [6]:

df_grouped = df.groupby('userId').agg({'rating': ['count'], 'timestamp': ['max', 'min']})
df_grouped_filtered = df_grouped[ df_grouped['rating']['count'] > 100 ].reset_index()
df_grouped_filtered['lifetime'] = df_grouped_filtered['timestamp']['max'] - df_grouped_filtered['timestamp']['min']

lifetime_mean = df_grouped_filtered.lifetime.mean() / 3600 / 24  
print('Cреднее время жизни пользователей, которые выставили более 100 оценок (сут.):', lifetime_mean)

# df_grouped_filtered.sort_values('lifetime', ascending=False)
df_grouped_filtered.sort_values(('rating', 'count'), ascending=False)


Cреднее время жизни пользователей, которые выставили более 100 оценок: 463.89476214829165


userId rating   timestamp               lifetime
            count         max         min           
208    547   2391  1476587644   974777109  501810535
217    564   1868   976224804   974708446    1516358
243    624   1735  1476616373  1019123866  457492507
2       15   1700  1469330735   997937239  471393496
23      73   1610  1476086345  1255500852  220585493
..     ...    ...         ...         ...        ...
77     217    104  1108160981  1108160000        981
12      36    104   853005800   847056510    5949290
242    623    103  1225258519  1225255417       3102
20      67    103   854715906   854711770       4136
95     257    103  1449941803  1338006929  111934874

[258 rows x 5 columns]

**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.


Дана статистика услуг перевозок клиентов компании по типам:

    rzd - железнодорожные перевозки
    auto - автомобильные перевозки
    air - воздушные перевозки
    client_base - адреса клиентов


In [67]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [68]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [69]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [70]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [71]:
first_table = rzd.merge(auto, 'outer', on='client_id').merge(air, 'outer', on='client_id')
first_table

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [78]:

second_table = client_base.merge(rzd, 'left', on='client_id')\
.merge(auto, 'left', on='client_id').merge(air, 'left', on='client_id')

second_table

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
1. У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
2. Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
3. Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Предполагаю, что нужно завести столбец, взяв среднее заначение по координатам каждого user_id за какой-то период времени, при этом удалив выбросы; либо определить моды (например, если пользователь заходит на сайт из дома и с работы) или что там еще есть в статистике полезного :) Просто брать среднее опасно тем, что вместо реальных мест нахождения пользователя можем получить какую-то среднюю координату места, где он никогда не был.
Если по такой таблице построить визуализацию на карте, то можно увидеть, например, где чаще совершаются покупки или частоту визитов, и, возможно, открыть в этом районе пункт выдачи товаров.